# Bronze to silver transformation 
This notebook performs transformations on data form the Bronze data LakeHouse and saves the transformation toothe silver Lakehouse. 

In [6]:
from pyspark.sql.functions import round, col, dayofmonth, dayofyear, month, to_date, quarter, year, substring, when, regexp_replace, cast

StatementMeta(, eef52393-47fc-4099-a68c-ff698785a9e4, 8, Finished, Available, Finished)

In [7]:
# define the path to the broze lakhouse 
bronze_table_path = "abfss://WINDPOWERGENERATION@onelake.dfs.fabric.microsoft.com/LH_BRONZE.Lakehouse/Tables/wind_power_production"

# Load the wind_power_production table into a DataFrame 
df = spark.read.format("delta").load(bronze_table_path)


StatementMeta(, eef52393-47fc-4099-a68c-ff698785a9e4, 9, Finished, Available, Finished)

In [8]:
df_transformed = (df
    .withColumn("wind_speed", round(col("wind_speed").cast("double"), 2))
    .withColumn("energy_produced", round(col("energy_produced").cast("double"), 2))
    .withColumn("day", dayofmonth(col('date')))
    .withColumn('month', month(col('date')))
    .withColumn('quarter', quarter(col('date')))
    .withColumn('year', year(col('date')))
    .withColumn('time', regexp_replace(col('time'), "-", ":"))
    .withColumn('hour_of_day', substring(col('time'), 1, 2).cast('int'))
    .withColumn('minute_of_hour', substring(col('time'), 4, 2).cast('int'))
    .withColumn('second_of_minute', substring(col('time'), 7, 2).cast('int'))
    .withColumn('time_period', when ((col('hour_of_day') >= 5) & (col('hour_of_day') < 12), "Morning")
                                .when ((col('hour_of_day') >= 12) & (col('hour_of_day') < 17), "Afternoon")
                                .when ((col('hour_of_day') >= 17) & (col('hour_of_day') < 21), "Evening")
                                .otherwise('Night')
    )



) 

StatementMeta(, eef52393-47fc-4099-a68c-ff698785a9e4, 10, Finished, Available, Finished)

In [9]:
#display(df_transformed)

StatementMeta(, eef52393-47fc-4099-a68c-ff698785a9e4, 11, Finished, Available, Finished)

In [10]:
# Path to the wind_production-_table in the silver Lakehouse
silver_table_path = "abfss://WINDPOWERGENERATION@onelake.dfs.fabric.microsoft.com/LH_SILVER.Lakehouse/Tables/wind_power_production"

# Save the transformed table in the silver Lakehouse
df_transformed.write.format("delta").mode("overwrite").save(silver_table_path)
df_transformed.write.format("delta").mode("append").save(silver_table_path)

StatementMeta(, eef52393-47fc-4099-a68c-ff698785a9e4, 12, Finished, Available, Finished)